<a href="https://colab.research.google.com/github/nano-bot01/Movie-Recommendation-System-for-Hollywoord-movies/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation system 

## Importing the dependencies

In [142]:
import numpy as np
import pandas as pd
import difflib
from scipy.sparse import find

from sklearn.feature_extraction.text import TfidfVectorizer 
# for transforming the textual data to numerical feature vector to find present similarity values 
from sklearn.metrics.pairwise import cosine_similarity

## Dataset Loading and Preprocessing


In [143]:
# loading the dataset

dataset = pd.read_csv('movies.csv')

In [144]:
dataset.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [145]:
dataset.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [146]:
# columns in dataset

dataset.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [147]:
# shape of dataset frame

dataset.shape

(4803, 24)

In [148]:
# information about dataset

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [149]:
# checking for null values in dataset

dataset.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [150]:
# Total count of null values

dataset.isnull().sum().sum()

4454

In [151]:
# changing the title of movies to lowercase 

dataset['original_title'] = dataset['original_title'].str.lower()
print(dataset['original_title'].head())

0                                      avatar
1    pirates of the caribbean: at world's end
2                                     spectre
3                       the dark knight rises
4                                 john carter
Name: original_title, dtype: object


In [152]:
selected_features = ['genres', 'keywords', 'tagline', 'cast','runtime', "director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'runtime', 'director']


In [153]:
# replacing the null values with null string

for features in selected_features :
  dataset[features] = dataset[features].fillna('')

### Combining the selected features 

In [154]:
dataset['runtime'] = dataset['runtime'].astype(str)

In [155]:
combined_features = dataset['genres']+''+dataset['keywords']+''+dataset['tagline']+''+dataset['cast']+''+dataset['runtime']+''+dataset['director']

In [156]:
combined_features.head()

0    Action Adventure Fantasy Science Fictioncultur...
1    Adventure Fantasy Actionocean drug abuse exoti...
2    Action Adventure Crimespy based on novel secre...
3    Action Crime Drama Thrillerdc comics crime fig...
4    Action Adventure Science Fictionbased on novel...
dtype: object

In [157]:
print(combined_features )

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


Converting the text data into features vector

In [158]:
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features )

In [159]:
print(feature_vector)

  (0, 4552)	0.16734937963693747
  (0, 393)	0.15504335673953543
  (0, 21698)	0.27051253672504794
  (0, 17579)	0.15504335673953543
  (0, 15518)	0.22094852179855456
  (0, 24229)	0.15973023366036612
  (0, 27504)	0.19398771440694682
  (0, 23429)	0.1996024624457141
  (0, 22454)	0.21286879022284044
  (0, 28486)	0.19529837407764436
  (0, 28127)	0.231932631287158
  (0, 22488)	0.15468316720695074
  (0, 19526)	0.25802589246546814
  (0, 19131)	0.08584667386065184
  (0, 28106)	0.12465726093125658
  (0, 25157)	0.07426699871794516
  (0, 23740)	0.27051253672504794
  (0, 5649)	0.242294581151395
  (0, 27294)	0.1290018968416829
  (0, 23878)	0.33257469678927243
  (0, 11168)	0.16330477529634707
  (0, 5428)	0.21533377375978727
  (0, 10279)	0.27051253672504794
  (0, 22804)	0.09844844294204591
  (0, 9910)	0.11661911728079709
  :	:
  (4801, 28307)	0.3066610964024391
  (4801, 28459)	0.29250586328316236
  (4801, 9092)	0.25415209931477006
  (4801, 191)	0.22995356846565448
  (4801, 1726)	0.1885083079472367
  (4801

### Cosine Similiarity 

In [160]:
# similarity confidence value using cosine similiarity

similarity = cosine_similarity(feature_vector)


In [161]:
print(similarity)

[[1.         0.06647634 0.01447541 ... 0.         0.         0.        ]
 [0.06647634 1.         0.0269466  ... 0.01195326 0.         0.        ]
 [0.01447541 0.0269466  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01195326 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [162]:
print(similarity.shape)
# compare with all the other movies present there

(4803, 4803)


In [163]:
# creating a list with all the moviee names in dataset

list_of_all_titles = dataset['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

## Recommedation

In [164]:
# getting the name as input for recommendation

movie_name = input('Enter your favourite movie name : ')

Enter your favourite movie name : batman


In [165]:
# finding the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Batman', 'Batman', 'Catwoman']


In [166]:
close_match = find_close_match[0]
print(close_match)

Batman


In [167]:
# Index of movie with title and data

index_of_movie = dataset[dataset.title == close_match]['index'].values[0]
# we are finding the index of movie by comparing the title of each movie with closly matched string and returing its index 
print(index_of_movie)

1359


In [168]:
# finding similariy score 

score = list(enumerate(similarity[index_of_movie]))
print(score)

[(0, 0.021851037696659643), (1, 0.03593985514418977), (2, 0.0), (3, 0.10476202307529849), (4, 0.01938388371197286), (5, 0.051577214342921895), (6, 0.0), (7, 0.0), (8, 0.034371763880017905), (9, 0.03325112233435061), (10, 0.10620688593062769), (11, 0.0), (12, 0.012128998054458755), (13, 0.009365337111072606), (14, 0.11464582769726374), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.038890666883741994), (19, 0.01716999129218622), (20, 0.0), (21, 0.0), (22, 0.009345232374230407), (23, 0.016907746765414424), (24, 0.02806035739976732), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.04799426718742124), (31, 0.019231381652176218), (32, 0.08699479087593497), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.019755908102682896), (37, 0.012725445257004999), (38, 0.0), (39, 0.023873642697138403), (40, 0.03221578676835574), (41, 0.03163150437339245), (42, 0.019971835298307865), (43, 0.009708577386352474), (44, 0.0), (45, 0.0), (46, 0.04861063548275448), (47, 0.0), (48, 0.0), (49, 0.00852485577154264

In [169]:
len(score)

4803

In [170]:
# finding movies having higher similarity score 

sorted_list = sorted(score, key = lambda x:x[1], reverse = True)
# sort based on non increasing order, key is based on first index of score in list element which is score in (index, score)
print(sorted_list)

[(1359, 1.0000000000000002), (428, 0.2575585778991273), (210, 0.1716372010478882), (2530, 0.1411653239060589), (473, 0.1358656645749242), (119, 0.12376598451253487), (14, 0.11464582769726374), (10, 0.10620688593062769), (1035, 0.10576377885973205), (278, 0.10521984519201324), (3, 0.10476202307529849), (1296, 0.1037079546259157), (1017, 0.10262332281551638), (1512, 0.10242277561315771), (1477, 0.10055542763657595), (1474, 0.09923945577343263), (65, 0.0991942594252579), (813, 0.09881678399021229), (2685, 0.09573384639209302), (438, 0.09473335506171368), (2108, 0.09185051583639256), (870, 0.09146608039861076), (1803, 0.09018777855101012), (2740, 0.08851613586238877), (32, 0.08699479087593497), (1141, 0.0866387775423049), (303, 0.0865418001565482), (215, 0.08484337292746223), (1076, 0.08463237514521435), (2365, 0.08461331586328377), (299, 0.08446407760187524), (4267, 0.08427592313047402), (1085, 0.08417190685716049), (2433, 0.08380292082614624), (583, 0.08247045824256187), (2826, 0.0820342

In [171]:
print(sorted_list[0], sorted_list[1], sorted_list[2], sorted_list[3], sorted_list[4])
# top five score with index 

(1359, 1.0000000000000002) (428, 0.2575585778991273) (210, 0.1716372010478882) (2530, 0.1411653239060589) (473, 0.1358656645749242)


In [172]:
# printing the name of simila rmovies based on the index

print("Movies Results : \n")

i = 1

for movie in sorted_list:
  index = movie[0]
  title_of_movie = dataset[dataset.index == index]['title'].values[0]
  if(i<=5):
    print(i, '.', title_of_movie)
    i += 1

Movies Results : 

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . Beetlejuice
5 . Mars Attacks!


## Recommendation function 

In [173]:
def recommend(name):
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  
  try:
    close_match = find_close_match[0]
  except IndexError:
    close_match = 'null'
    print("No result found with this title")
    return 
  index_of_movie = dataset[dataset.title == close_match]['index'].values[0]
  score = list(enumerate(similarity[index_of_movie]))
  sorted_list = sorted(score, key = lambda x:x[1], reverse = True)


  print("Movies Results : \n")

  i = 1

  for movie in sorted_list:
    index = movie[0]
    title_of_movie = dataset[dataset.index == index]['title'].values[0]
    if(i<=5):
      print(i, '. ', title_of_movie)
      i += 1

In [174]:
movie_name = input('Enter your favourite movie name : ')

recommend(movie_name)

Enter your favourite movie name : bad boy
Movies Results : 

1 .  Bad Boys
2 .  Kiss of the Dragon
3 .  Bad Boys II
4 .  Wing Commander
5 .  The Good Thief
